In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from pprint import pprint
# from gensim.models import KeyedVectors

print(tf.__version__)

1.4.0


In [2]:
def binarize_pos_neg(score):
    if score >= 5.0:
        score = 1
    else:
        score = 0
    return score

In [3]:
pos_neg_data_pre = pd.read_csv('pos_neg_labeling_7scaled.tsv', delimiter='\t')
pos_neg_data_pre.rename(columns={'긍정/부정 척도': 'sentiment'}, inplace=True)
neutral_sentiment = pos_neg_data_pre[pos_neg_data_pre['sentiment'] == 4.0]
neutral_sentiment_idx = neutral_sentiment.index
pos_neg_data_pre_wo_neutral = pos_neg_data_pre.drop(neutral_sentiment_idx).reset_index(drop=True)
pos_neg_data_pre_wo_neutral.drop(['명확도', 'Emojiness'], axis=1, inplace=True)
pos_neg_data = pos_neg_data_pre_wo_neutral.copy()
pos_neg_data['sentiment'] = pos_neg_data['sentiment'].apply(binarize_pos_neg)

sentences = pos_neg_data['queries']
sentence_splitted = [sentence.split(' ') for sentence in sentences]

In [4]:
word_to_id = {}
id_to_word = {}

for sentence in sentence_splitted:
    for word in sentence:
        if word not in word_to_id:
            word_to_id[word] = len(word_to_id)

for word, word_id in word_to_id.items():
    id_to_word[word_id] = word

print(word_to_id['설거지하고'])
print(id_to_word[word_to_id['설거지하고']])

2435
설거지하고


In [5]:
sentences_encoded = []

for sentence in sentence_splitted:
    sentence_encoded = []
    for word in sentence:
        sentence_encoded.append(word_to_id[word])
    sentences_encoded.append(sentence_encoded)

sentences_encoded[:10]

[[0, 1, 2, 3],
 [4, 5, 2, 6],
 [7, 8],
 [9, 3],
 [10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20],
 [21],
 [4, 22, 23],
 [24, 8],
 [25]]

In [6]:
vocab_size = 605918
embedding_dim = 256

w = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]), trainable=False, name='W')
sentences = tf.placeholder(tf.int32, shape=[None, None])

config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)

embedding_saver = tf.train.Saver({'lm/softmax/W': w})
embedding_saver.restore(sess, '/media/scatter/scatterdisk/elmo_ckpt_0831_1233/model_batch4700.ckpt-4700')
embedding_lookup = tf.nn.embedding_lookup(w, sentences)

INFO:tensorflow:Restoring parameters from /media/scatter/scatterdisk/elmo_ckpt_0831_1233/model_batch4700.ckpt-4700


In [7]:
tf.global_variables_initializer()

<tf.Operation 'init' type=NoOp>

In [35]:
embeddings = []

for sentence in sentences_encoded:
    sentence_embedding = sess.run(embedding_lookup, feed_dict={sentences: [sentence]})
    embeddings.append(sentence_embedding)

In [44]:
embeddings[-1].shape

(1, 9, 256)

In [2]:
# works fine

config = tf.ConfigProto(allow_soft_placement=True)

with tf.Session(config=config) as sess:
    saver = tf.train.import_meta_graph('/media/scatter/scatterdisk/elmo_ckpt_0831_1405/model_batch4700.ckpt-4700.meta')
    saver.restore(sess, '/media/scatter/scatterdisk/elmo_ckpt_0831_1405/model_batch4700.ckpt-4700')

INFO:tensorflow:Restoring parameters from /media/scatter/scatterdisk/elmo_ckpt_0831_1405/model_batch4700.ckpt-4700
